In [131]:
#DO NOT RUN Unit Tests or Testing Blocks
import pandas as pd
import numpy as np
import random as rd
import math
from random import sample
import os
import datetime
# import sys
# !{sys.executable} -m pip install surprise
from surprise import AlgoBase, Dataset, SVD, Reader, accuracy, SVDpp
from surprise.model_selection import cross_validate, KFold, train_test_split
from collections import defaultdict

In [ ]:
class SVDImpl(AlgoBase):
    def __init__(self, learning_rate, n_epochs, n_factors):
        
        self.lr = learning_rate  # learning rate for SGD
        self.n_epochs = n_epochs  # number of iterations of SGD
        self.n_factors = n_factors  # number of factors
        self.lr_all = .005
        self.reg_all = .02
        AlgoBase.__init__(self)
        
    def fit(self, trainset):
        AlgoBase.fit(self, trainset)
        '''Learn the vectors p_u and q_i with SGD'''
        
        print('Fitting data with SGD...')
        #Initialize the user and item biases
        bu = np.zeros(trainset.n_users)
        bi = np.zeros(trainset.n_items)
        # Randomly initialize the user and item factors.
        pu = np.random.normal(0, .1, (trainset.n_users, self.n_factors))
        qi = np.random.normal(0, .1, (trainset.n_items, self.n_factors))
        lr_bu, lr_bi, lr_pu, lr_qi = self.lr_all, self.lr_all, self.lr_all, self.lr_all
        reg_bu, reg_bi, reg_pu, reg_qi = self.reg_all, self.reg_all, self.reg_all, self.reg_all
        # SGD procedure
        for _ in range(self.n_epochs):
            for u, i, r_ui in trainset.all_ratings():
                dot = 0  # <q_i, p_u>
                for f in range(self.n_factors):
                    dot += qi[i, f] * pu[u, f]
                err = r_ui - (trainset.global_mean + bu[u] + bi[i] + dot)

                # update biases
                bu[u] += lr_bu * (err - reg_bu * bu[u])
                bi[i] += lr_bi * (err - reg_bi * bi[i])

                # update factors
                for f in range(self.n_factors):
                    puf = pu[u, f]
                    qif = qi[i, f]
                    pu[u, f] += lr_pu * (err * qif - reg_pu * puf)
                    qi[i, f] += lr_qi * (err * puf - reg_qi * qif)
        
        self.bu = np.asarray(bu)
        self.bi = np.asarray(bi)
        self.pu = np.asarray(pu)
        self.qi = np.asarray(qi)

    def estimate(self, u, i):
        '''Return the estmimated rating of user u for item i.'''
        
        known_user = self.trainset.knows_user(u)
        known_item = self.trainset.knows_item(i)

        est = self.trainset.global_mean

        if known_user:
            est += self.bu[u]

        if known_item:
            est += self.bi[i]

        if known_user and known_item:
            est += np.dot(self.qi[i], self.pu[u])
        return est

In [ ]:
r_cols = ['user_id', 'movie_id', 'rating','timeStamp']
ratings = pd.read_csv('ratings.csv', header=None,skiprows=1,sep=',', names=r_cols)
data = pd.DataFrame(ratings,columns = ['user_id','movie_id','timeStamp'])
tsDict = dict()
#setting dict to access timestamps of ratings
for row in data.iterrows():
    tsDict[(row[1]['user_id'], row[1]['movie_id'])] = row[1]['timeStamp']
data_ind = data.set_index(['user_id','movie_id'])
meanDate= np.nanmean(data_ind)
avgtime_byuser = data.groupby(by=data.user_id).timeStamp.mean()
avgTimeU = dict()
for uid, avgts in avgtime_byuser.items():
    avgTimeU[uid] = avgts

print("DONE")

In [ ]:


class timeSVDImplI(AlgoBase):
    def __init__(self, learning_rate, n_epochs, n_factors):
        
        self.lr = learning_rate  # learning rate for SGD
        self.n_epochs = n_epochs  # number of iterations of SGD
        self.n_factors = n_factors  # number of factors
        self.lr_all = .005
        self.reg_all = .02
        self.numBins = 23  # 23 bins yearly from 1996 to 2018
        self.meanDate = meanDate
        self.B = .00005
        AlgoBase.__init__(self)
       
        
        
    def fit(self, trainset):
        AlgoBase.fit(self, trainset)
        '''Learn the vectors p_u and q_i with SGD'''
        
        print('Fitting data with SGD...')
        #Initialize the user and item biases
        bu = np.zeros(trainset.n_users, dtype=np.longdouble)
        au = np.random.normal(0, .1, (trainset.n_users))
        au = np.zeros(trainset.n_users, dtype=np.longdouble)
        bi = np.zeros(trainset.n_items, dtype=np.longdouble)
        biBin = [np.copy(bi)]*23
        # but = [np.copy(bu)]*40
        but = [dict()]*trainset.n_users
        
        # Randomly initialize the user and item factors.
        pu = np.random.normal(0, .1, (trainset.n_users, self.n_factors))
        qi = np.random.normal(0, .1, (trainset.n_items, self.n_factors))
        lr_bu, lr_bi, lr_pu, lr_qi = self.lr_all, self.lr_all, self.lr_all, self.lr_all
        reg_bu, reg_bi, reg_pu, reg_qi = self.reg_all, self.reg_all, self.reg_all, self.reg_all
        
        # SGD procedure
        for _ in range(self.n_epochs):
            print("Going through epoch...")
            for u, i, r_ui in trainset.all_ratings():
                
                dot = 0  
                for f in range(self.n_factors):
                    dot += qi[i, f] * pu[u, f]
                u_rawid = int(trainset.to_raw_uid(u))
                i_rawid = int(trainset.to_raw_iid(i))
                
        
                # diffDaySign = np.sign(currTs-avgTimeU[u_rawid])
                # devu_t = diffDaySign * pow(abs((datetime.datetime.fromtimestamp(currTs)- \
                #                        datetime.datetime.fromtimestamp(avgTimeU[u_rawid]))).days, self.B)
              
                # if currTsF.days not in but[u]:
                #     but[u][currTsF.days] = 0
                    
                err = r_ui - (trainset.global_mean + bu[u] + bi[i] + dot)
                                # au[u]*devu_t) # + but[u][currTsF.days])
                # needed for when we're predicting with unknown timestamps
                if (u_rawid,i_rawid) in tsDict:
                    currTs = tsDict[(u_rawid,i_rawid)]
                    # currTsF = (datetime.datetime.fromtimestamp(currTs)- datetime.datetime(1996,3,29))
                    binNum = datetime.datetime.fromtimestamp(currTs).year-1996
                    err -= biBin[binNum][i]
                    biBin[binNum][i] += lr_bi * (err - reg_bi * biBin[binNum][i])
                # update biases
                
                # movie bias
                #bi(t) = bi + bi,Bin(t)
                bi[i] += lr_bi * (err - reg_bi * bi[i]) 
                
                
                # user bias
                #b u (t) = bu + αu · devu(t) 
                bu[u] += lr_bu * (err - reg_bu * bu[u])
                # au[u] += lr_bu * (err*devu_t - reg_bu * au[u]) # SEE SGD CALCULATION/DERIVATIVE
                
                # but[u][currTsF.days] = lr_bu * (err - reg_bu * but[u][currTsF.days])
                
                # update factors
                for f in range(self.n_factors):
                    puf = pu[u, f]
                    qif = qi[i, f]
                    pu[u, f] += lr_pu * (err * qif - reg_pu * puf)
                    qi[i, f] += lr_qi * (err * puf - reg_qi * qif)
        
        self.bu = np.asarray(bu)
        self.bi = np.asarray(bi)
        self.biBin = np.asarray(biBin)
        self.but = np.asarray(but)
        
        self.au = np.asarray(au)
        self.pu = np.asarray(pu)
        self.qi = np.asarray(qi)
        
    def estimate(self, u, i):
        '''Return the estmimated rating of user u for item i.'''
        
        known_user = self.trainset.knows_user(u)
        known_item = self.trainset.knows_item(i)

        est = self.trainset.global_mean
        
        if known_user:
            est += self.bu[u]
            

        if known_item:
            est += self.bi[i]

        if known_user and known_item:
            u_rawid = int(self.trainset.to_raw_uid(u))
            i_rawid = int(self.trainset.to_raw_iid(i))
            if (u_rawid,i_rawid) in tsDict:
                currTs = tsDict[(u_rawid,i_rawid)]
                if currTs == 0:
                    print("FOUND")
                currTsF = (datetime.datetime.fromtimestamp(currTs)- datetime.datetime(1996,3,29))
                binNum = datetime.datetime.fromtimestamp(currTs).year-1996
                #BiBin(t)
                est += self.biBin[binNum][i]
            
            #au*devu_t(t)
            # diffDaySign = np.sign(currTs-avgTimeU[u_rawid])
            # devu_t = diffDaySign * pow(abs((datetime.datetime.fromtimestamp(currTs)- \
            #     datetime.datetime.fromtimestamp(avgTimeU[u_rawid]))).days, self.B)
            
            # est += self.au[u]*devu_t
            
            #jank hack, why is currTsF.days not a valid key sometimes?
            # if currTsF.days in self.but[u]:
            #     est += self.but[u][currTsF.days]
            est += np.dot(self.qi[i], self.pu[u])
            
        return est

In [ ]:


class timeSVDImplUI(AlgoBase):
    def __init__(self, learning_rate, n_epochs, n_factors):
        
        self.lr = learning_rate  # learning rate for SGD
        self.n_epochs = n_epochs  # number of iterations of SGD
        self.n_factors = n_factors  # number of factors
        self.lr_all = .005
        self.reg_all = .02
        self.numBins = 23  # 23 bins yearly from 1996 to 2018
        self.meanDate = meanDate
        self.B = .00005
        AlgoBase.__init__(self)
       
        
        
    def fit(self, trainset):
        AlgoBase.fit(self, trainset)
        '''Learn the vectors p_u and q_i with SGD'''
        
        print('Fitting data with SGD...')
        #Initialize the user and item biases
        bu = np.zeros(trainset.n_users, dtype=np.longdouble)
        au = np.random.normal(0, .1, (trainset.n_users))
        au = np.zeros(trainset.n_users, dtype=np.longdouble)
        bi = np.zeros(trainset.n_items, dtype=np.longdouble)
        biBin = [np.copy(bi)]*23
        # but = [np.copy(bu)]*40
        but = [dict()]*trainset.n_users
        
        # Randomly initialize the user and item factors.
        pu = np.random.normal(0, .1, (trainset.n_users, self.n_factors))
        qi = np.random.normal(0, .1, (trainset.n_items, self.n_factors))
        lr_bu, lr_bi, lr_pu, lr_qi = self.lr_all, self.lr_all, self.lr_all, self.lr_all
        reg_bu, reg_bi, reg_pu, reg_qi = self.reg_all, self.reg_all, self.reg_all, self.reg_all
        
        # SGD procedure
        for _ in range(self.n_epochs):
            print("Going through epoch...")
            for u, i, r_ui in trainset.all_ratings():
                
                dot = 0  
                for f in range(self.n_factors):
                    dot += qi[i, f] * pu[u, f]
                u_rawid = int(trainset.to_raw_uid(u))
                i_rawid = int(trainset.to_raw_iid(i))
                
              
                # if currTsF.days not in but[u]:
                    # but[u][currTsF.days] = 0
                    
                err = r_ui - (trainset.global_mean + bu[u] + bi[i] + dot) # + biBin[binNum][i] + \
                                # au[u]*devu_t) # + but[u][currTsF.days])
                
                # update biases
                if (u_rawid,i_rawid) in tsDict:
                    currTs = tsDict[(u_rawid,i_rawid)]
                    # currTsF = (datetime.datetime.fromtimestamp(currTs)- datetime.datetime(1996,3,29))
                    binNum = datetime.datetime.fromtimestamp(currTs).year-1996
                    err -= biBin[binNum][i]
                    
                    if u_rawid in avgTimeU: 
                        diffDaySign = np.sign(currTs-avgTimeU[u_rawid])
                        devu_t = diffDaySign * pow(abs((datetime.datetime.fromtimestamp(currTs)- \
                                               datetime.datetime.fromtimestamp(avgTimeU[u_rawid]))).days, self.B)
                        err -= au[u]*devu_t
                        
                    biBin[binNum][i] += lr_bi * (err - reg_bi * biBin[binNum][i])
                    if u_rawid in avgTimeU:
                        au[u] += lr_bu * (err*devu_t - reg_bu * au[u]) # SEE SGD CALCULATION/DERIVATIVE
                # movie bias
                #bi(t) = bi + bi,Bin(t)
                bi[i] += lr_bi * (err - reg_bi * bi[i]) 
                
                
                # user bias
                #b u (t) = bu + αu · devu(t) 
                bu[u] += lr_bu * (err - reg_bu * bu[u])
                
                
                # but[u][currTsF.days] = lr_bu * (err - reg_bu * but[u][currTsF.days])
                
                # update factors
                for f in range(self.n_factors):
                    puf = pu[u, f]
                    qif = qi[i, f]
                    pu[u, f] += lr_pu * (err * qif - reg_pu * puf)
                    qi[i, f] += lr_qi * (err * puf - reg_qi * qif)
        
        self.bu = np.asarray(bu)
        self.bi = np.asarray(bi)
        self.biBin = np.asarray(biBin)
        self.but = np.asarray(but)
        
        self.au = np.asarray(au)
        self.pu = np.asarray(pu)
        self.qi = np.asarray(qi)
        
    def estimate(self, u, i):
        '''Return the estmimated rating of user u for item i.'''
        
        known_user = self.trainset.knows_user(u)
        known_item = self.trainset.knows_item(i)

        est = self.trainset.global_mean
        
        if known_user:
            est += self.bu[u]
            

        if known_item:
            est += self.bi[i]

        if known_user and known_item:
            u_rawid = int(self.trainset.to_raw_uid(u))
            i_rawid = int(self.trainset.to_raw_iid(i))
            if (u_rawid,i_rawid) in tsDict:
                currTs = tsDict[(u_rawid,i_rawid)]
                currTsF = (datetime.datetime.fromtimestamp(currTs)- datetime.datetime(1996,3,29))
                binNum = datetime.datetime.fromtimestamp(currTs).year-1996
                #BiBin(t)
                est += self.biBin[binNum][i]
                if u_rawid in avgTimeU:
                    #au*devu_t(t)
                    diffDaySign = np.sign(currTs-avgTimeU[u_rawid])
                    devu_t = diffDaySign * pow(abs((datetime.datetime.fromtimestamp(currTs)- \
                        datetime.datetime.fromtimestamp(avgTimeU[u_rawid]))).days, self.B)
                    est += self.au[u]*devu_t
            
            #jank hack, why is currTsF.days not a valid key sometimes?
            # if currTsF.days in self.but[u]:
            #     est += self.but[u][currTsF.days]
            est += np.dot(self.qi[i], self.pu[u])
            
        return est

In [ ]:
# Load the movielens-100k dataset (download it if needed),
reader = Reader(line_format="user item rating timestamp", sep=",", skip_lines=1)

file_path = "/common/home/js2715/cs550/ratings.csv"
data = Dataset.load_from_file(file_path, reader=reader)

# We'll use the famous SVD algorithm.
algo = timeSVDImplUI(learning_rate=.01, n_epochs=10, n_factors=10)
# algo2 = SVDImpl(learning_rate=.01, n_epochs=10, n_factors=10)
# algo2 = SVD()
# Run 5-fold cross-validation and print results
cross_validate(algo, data, measures=["RMSE", "MAE"], cv=2, verbose=True)
# cross_validate(algo2, data, measures=["RMSE", "MAE"], cv=2, verbose=True)


Creation of new "proper" training and testing sets (refer to report for explanation)
Basically two fold, sorting by timestamp and taking proportion of ratings from each user

In [ ]:
r_cols = ['user_id', 'movie_id', 'rating','timeStamp']
ratings = pd.read_csv('ratings.csv', header=None,skiprows=1,sep=',', names=r_cols)
data = pd.DataFrame(ratings,columns = ['user_id','movie_id', 'rating', 'timeStamp'])
sortedData = data.sort_values(['user_id','timeStamp'],ascending=False).groupby('user_id')
training_data = pd.DataFrame(columns = ['user_id','movie_id', 'rating', 'timeStamp'])
testing_data = pd.DataFrame(columns = ['user_id','movie_id', 'rating', 'timeStamp'])
for key, item in sortedData:
    cutoff = int(len(item.index)*.2)
    df1 = item.iloc[:cutoff,:]
    df2 = item.iloc[cutoff:,:]
    for ind, row in df2.iterrows():
        training_data.loc[len(training_data)] = row
    for ind, row in df1.iterrows():
        testing_data.loc[len(testing_data)] = row
print("DONE")
   
    



In [ ]:
# training_data = training_data.drop(columns=['timeStamp'])
# testing_data = testing_data.drop(columns=['timeStamp'])
reader = Reader(rating_scale=(1, 5))
# The columns must correspond to user id, item id and ratings (in that order).
train_data = Dataset.load_from_df(training_data[['user_id','movie_id', 'rating']], reader)
train_data = train_data.build_full_trainset()
test_data = Dataset.load_from_df(testing_data[['user_id','movie_id', 'rating']], reader)

new_test = test_data.build_full_trainset().build_testset()

In [ ]:
algo = timeSVDImplUI(learning_rate=.01, n_epochs=10, n_factors=10)
# algo = SVDImpl(learning_rate=.01, n_epochs=10, n_factors=10)
# algo = SVD()
# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(train_data)
predictions = algo.test(new_test)

# Then compute RMSE
accuracy.rmse(predictions)
accuracy.mae(predictions)

Getting all of the top 10 recommendations for each user

In [ ]:
# Modified from https://github.com/NicolasHug/Surprise/blob/master/examples/top_n_recommendations.py 
#- Nicholas Hug's (Creator of Surprise library) implementation of top_n_recommendations
def get_top_n(predictions, n=10):
    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

data = Dataset.load_from_file(file_path, reader=reader)
trainset = data.build_full_trainset() 
algo = SVDImpl(learning_rate=.01, n_epochs=10, n_factors=10) 
algo.fit(trainset)

testset = trainset.build_anti_testset()
predictions = algo.test(testset)

top_n_SVDImpl = get_top_n(predictions, n=10)

# Print the recommended items for each user
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])

In [ ]:
data = Dataset.load_from_file(file_path, reader=reader)
trainset = data.build_full_trainset() 
algo = timeSVDImplI(learning_rate=.01, n_epochs=10, n_factors=10) 
algo.fit(trainset)

testset = trainset.build_anti_testset()
predictions = algo.test(testset)

top_n_SVDImplI = get_top_n(predictions, n=10)

# Print the recommended items for each user
print("SVDIMPLI TOP RECOMMENDATIONS")
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])


In [ ]:
data = Dataset.load_from_file(file_path, reader=reader)
trainset = data.build_full_trainset() 
algo = timeSVDImplUI(learning_rate=.01, n_epochs=10, n_factors=10) 
algo.fit(trainset)


testset = trainset.build_anti_testset()
predictions = algo.test(testset)

top_n_SVDImplUI = get_top_n(predictions, n=10)

# Print the recommended items for each user
print("SVDIMPLUI TOP RECOMMENDATIONS")
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])

In [132]:
#Modified from https://surprise.readthedocs.io/en/stable/FAQ.html#precision-recall-at-k-py 
def precision_recall_at_k(predictions, k=10, threshold=3.5):
    """Return precision and recall at k metrics for each user"""

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))
    
    precisions = dict()
    recalls = dict()
    ndcg = dict()
    for uid, user_ratings in user_est_true.items():
        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)
        
        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)
        
        dcg = sum((true_r*(1/(math.log2(ind+2)))) for ind, (_, true_r) in enumerate(user_ratings))
        dcg_max = sum((true_r*(1)) for ind, (_, true_r) in enumerate(user_ratings))
        ndcg[uid] = dcg/dcg_max
        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(
            ((true_r >= threshold) and (est >= threshold))
            for (est, true_r) in user_ratings[:k]
        )

        # Precision@K: Proportion of recommended items that are relevant
        # When n_rec_k is 0, Precision is undefined. We here set it to 0.

        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0

        # Recall@K: Proportion of relevant items that are recommended
        # When n_rel is 0, Recall is undefined. We here set it to 0.

        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0
        
    return precisions, recalls, ndcg


data = Dataset.load_builtin("ml-100k")

# algo = timeSVDImplUI(learning_rate=.01, n_epochs=10, n_factors=10)
algo = SVDpp()
trainset, testset = train_test_split(data, test_size=0.2)
algo.fit(trainset)
predictions = algo.test(testset)
precisions, recalls, ndcg = precision_recall_at_k(predictions, k=10, threshold=3.5)
total_prec = sum(prec for prec in precisions.values()) / len(precisions)
total_rec =sum(rec for rec in recalls.values()) / len(recalls)
total_ndcg = sum(val for val in ndcg.values()) / len(ndcg)
# Precision and recall can then be averaged over all users
print("Precision:", total_prec)
print("Recall:", total_rec)
print("F-Measure:",(2*total_prec*total_rec)/(total_prec+total_rec))
print("NDCG:", total_ndcg)
    
    


Precision: 0.7207170660196139
Recall: 0.5455292063587537
F-Measure: 0.6210043300603779
NDCG: 0.4470021892019521
